In [1]:
#!/usr/bin/env python

In [2]:
import os
import sys
if os.path.exists('/home/chieh/code/wPlotLib'):
	sys.path.insert(0,'/home/chieh/code/wPlotLib')
if os.path.exists('/home/chieh/code/wuML'):
	sys.path.insert(0,'/home/chieh/code/wuML')

In [3]:
import wuml
import numpy as np
import torch
import torch
import torch.nn as nn

In [4]:
def costFunction(x, x̂, ẙ, y, ŷ, ind):	
#	x -> encoder -> x̂
#	x̂ -> encoder_linear_output -> ẙ	
#	x̂ -> decoder -> ŷ	
#	possible autoencoder objective λ could be 0
#	loss = (x - ŷ)ᒾ + λ * objective(ẙ, y)
#
#	This function can return 1 value or 3 values in a list
#	if return 1 value, just the loss
#	if return 3 values, [total_loss, reconstruction_loss, extra network from ẙ loss]
#
#	In this example, we perform both reconstruction and CE loss
#
	CE_loss = nn.CrossEntropyLoss() #weird pytorch, dim of y is 1, and ŷ is 20x3
	R = torch.sum((x - ŷ) ** 2)/(128*13)	#scaled by batch size times data dimension
	CE = CE_loss(ẙ, y)
	loss = R + CE
	return [loss, R, CE]

In [5]:
def costFunction2(x, x̂, ẙ, y, ŷ, ind):	
#	Example of just return 1 value, this is just a regular autoencoder
	return torch.sum((x - ŷ) ** 2)

In [6]:
data = wuml.wData(xpath='../../data/wine.csv', ypath='../../data/wine_label.csv', 
					preprocess_data='center and scale', batch_size=128, label_type='discrete')

In [7]:
AE = wuml.autoencoder(12, data, default_depth=2, costFunction=costFunction, # costFunction and costFunction2 both works
						max_epoch=2000, encoder_output_weight_structure=[(3,'none')] ) 
AE.fit()

	epoch: 2000, Total Loss/dimension: 0.0077, Reconstruct loss: 0.0076, Objective loss: 0.0001, Learning Rate: 0.00006250

Notice that the reconstruction did a reasonable job

In [8]:
ŷ = AE(data)
wuml.jupyter_print(ŷ[0:5,0:10])
wuml.jupyter_print(data.X[0:5,0:10])

tensor([[ 1.5150, -0.5602,  0.2031, -1.0998,  1.9174,  0.8163,  1.0218, -0.6630,  1.1992,  0.2587],
        [ 0.2541, -0.4794, -0.8487, -2.4189,  0.0220,  0.5568,  0.7586, -0.8353, -0.5799, -0.2628],
        [ 0.2019,  0.0295,  1.1201, -0.2598,  0.0852,  0.8167,  1.2073, -0.5111,  2.1294,  0.2874],
        [ 1.7670, -0.2610,  0.3909, -0.7634,  1.0001,  2.1782,  1.9844, -0.9473,  0.9935,  1.2145],
        [ 0.2910,  0.2236,  1.8294,  0.4521,  1.2841,  0.7960,  0.6811,  0.2337,  0.4052, -0.3322]], device='cuda:0', grad_fn=<SliceBackward0>)
[[ 1.5186 -0.5622  0.2321 -1.1696  1.9139  0.809   1.0348 -0.6596  1.2249  0.2517]
 [ 0.2463 -0.4994 -0.828  -2.4908  0.0181  0.5686  0.7336 -0.8207 -0.5447 -0.2933]
 [ 0.1969  0.0212  1.1093 -0.2687  0.0884  0.809   1.2155 -0.4984  2.136   0.269 ]
 [ 1.6915 -0.3468  0.4879 -0.8093  0.9309  2.4914  1.4665 -0.9819  1.0322  1.1861]
 [ 0.2957  0.2277  1.8404  0.4519  1.282   0.809   0.6634  0.2268  0.4014 -0.3193]]


This is the bottleneck output which is d=12

In [11]:
x̂ = AE.reduce_dimension(data, output_type='wData')
wuml.jupyter_print(x̂.shape)

This is the objective network output 

In [12]:
ẙ = AE.objective_network(data)

Here we use the bottleneck to perform SVM classification

In [13]:
cf = wuml.classification(x̂, classifier='SVM')
wuml.jupyter_print(cf.result_summary(print_out=False))

,classifier,Train,Test
,SVM,1.0,1.0


Here we use the objective network output to perform LogisticRegression classification

In [14]:
cf = wuml.classification(ẙ, classifier='LogisticRegression')
wuml.jupyter_print(cf.result_summary(print_out=False))

,classifier,Train,Test
,LogisticRegression,1.0,1.0
